In [ ]:
%%capture
# Install necessary libraries
!pip install transformers datasets peft bitsandbytes accelerate -q
!pip install huggingface_hub
!pip install torch transformers peft datasets accelerate bitsandbytes


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from huggingface_hub import login

# Login to Hugging Face (Replace with your own token)
login(token="")

# Load the base LLaMA model
model_name = "meta-llama/Llama-3.2-3B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

# **Tokenizer Yükle**
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Pad token tanımla

# **Modeli Yükle**
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"  # GPU'yu otomatik seç
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# **1️⃣ W&B Kapat**
os.environ["WANDB_DISABLED"] = "true"

# **2️⃣ Model ve Tokenizer Tanımla**

# **4️⃣ LoRA Konfigürasyonu**
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# **5️⃣ Veri Setini Yükle**
train_data_path = "/content/drive/MyDrive/fine_tune_data.json"
dataset = load_dataset("json", data_files=train_data_path, split="train")

# **6️⃣ Dataset Formatını Düzenle**
def format_dataset(example):
    formatted_text = f"### Instruction: {example['instruction']}\n### Response: {example['response']}"
    return {"text": formatted_text}

dataset = dataset.map(format_dataset)

# **7️⃣ Tokenize Et**
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()  # Labels ayarla
    return tokenized

dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.remove_columns(["instruction", "response", "context"])

# **8️⃣ Veri Hazırlayıcı**
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

# **9️⃣ Eğitim Parametreleri**
training_args = TrainingArguments(
    output_dir="./llama_finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
    report_to="none"
)

# **🔟 Trainer Tanımla**
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    data_collator=data_collator
)

# **📌 Eğitimi Başlat**
trainer.train()




Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.372400,0.351030
2,0.406700,0.281897
3,0.258400,0.257437
4,0.182700,0.249469
5,0.356900,0.247021


TrainOutput(global_step=520, training_loss=0.7660762481964551, metrics={'train_runtime': 686.9918, 'train_samples_per_second': 1.514, 'train_steps_per_second': 0.757, 'total_flos': 9020216243650560.0, 'train_loss': 0.7660762481964551, 'epoch': 5.0})

In [ ]:
# 🔹 LoRA Adaptörünü Kaydet
model.save_pretrained("llama_finetuned")

# 🔹 Tokenizer'ı Kaydet
tokenizer.save_pretrained("llama_finetuned")

# 🔹 Temel Modeli de Kaydetmek İçin (Önemli!)
base_model = model.base_model  # Ana model referansı
base_model.save_pretrained("llama_finetuned/base_model")

print("Model başarıyla kaydedildi!")


Model başarıyla kaydedildi!


In [ ]:
import shutil

# Model klasörünü sıkıştır (ZIP formatında)
shutil.make_archive("llama_finetuned", 'zip', "llama_finetuned")

print("Model sıkıştırıldı: llama_finetuned.zip")


Model sıkıştırıldı: llama_finetuned.zip


In [ ]:
from google.colab import drive

# Drive'ı bağla
drive.mount('/content/drive')

# ZIP dosyasını Drive'a taşı
shutil.move("llama_finetuned.zip", "/content/drive/MyDrive/")

print("Model Google Drive'a kaydedildi!")
from google.colab import files

files.download("llama_finetuned.zip")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model Google Drive'a kaydedildi!


FileNotFoundError: Cannot find file: llama_finetuned.zip

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Tokenizer'ı yükle
tokenizer = AutoTokenizer.from_pretrained("llama_finetuned")

# Temel Modeli yükle
base_model = AutoModelForCausalLM.from_pretrained("llama_finetuned/base_model")

# LoRA Adaptörünü Uygula
model = PeftModel.from_pretrained(base_model, "llama_finetuned")

print("Eğitilmiş model başarıyla yüklendi!")


In [ ]:
import shutil
from google.colab import files

# **1️⃣ Modeli Kaydet**
output_dir = "./llama_finetuned"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# **2️⃣ Modeli Google Drive'a Kaydet**
drive_path = "/content/drive/MyDrive/llama_finetuned"
shutil.copytree(output_dir, drive_path)



'/content/drive/MyDrive/llama_finetuned'

In [ ]:
# **3️⃣ Model Dosyalarını Bilgisayara İndir**
shutil.make_archive("llama_finetuned", 'zip', output_dir)
files.download("llama_finetuned.zip")

print(f"Model başarıyla {output_dir} klasörüne, Google Drive'daki {drive_path} klasörüne kaydedildi ve bilgisayara indirilmek üzere sıkıştırıldı.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model başarıyla ./llama_finetuned klasörüne, Google Drive'daki /content/drive/MyDrive/llama_finetuned klasörüne kaydedildi ve bilgisayara indirilmek üzere sıkıştırıldı.


In [ ]:
full_model_path = "./llama_finetuned_full"

# Eğer LoRA kullanılıyorsa adaptörleri birleştir ve kaldır
if hasattr(model, "merge_and_unload"):
    model = model.merge_and_unload()

# Modeli kaydet
model.save_pretrained(full_model_path)
tokenizer.save_pretrained(full_model_path)

print(f"Eğitilmiş model '{full_model_path}' klasörüne kaydedildi!")


Eğitilmiş model './llama_finetuned_full' klasörüne kaydedildi!


In [ ]:
import shutil
from google.colab import files

# Model klasörünü sıkıştır
shutil.make_archive("llama_finetuned_full", 'zip', "./llama_finetuned_full")

# Zip dosyasını bilgisayara indir
files.download("llama_finetuned_full.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil
shutil.make_archive("llama_finetuned_full", 'zip', "./llama_finetuned_full")

from google.colab import files
files.download("llama_finetuned_full.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil

# Modeli Google Drive içine kaydet
shutil.copytree("./llama_finetuned", "/content/drive/MyDrive/llama_finetuned")


FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/llama_finetuned'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Kaydedilen modelin klasörü
model_path = "./llama_finetuned_full"

# Model ve tokenizer'ı yükle
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

print("Eğitilmiş model başarıyla yüklendi!")


In [ ]:
# Tam modeli kaydet
merged_model_path = "/content/drive/MyDrive/llama_finetuned_full"
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)

print(f"Tam model {merged_model_path} dizinine kaydedildi! 🚀")

Tam model /content/drive/MyDrive/llama_finetuned_full dizinine kaydedildi! 🚀


In [ ]:
import shutil
shutil.make_archive("llama_finetuned", 'zip', "./llama_finetuned")

from google.colab import files
files.download("llama_finetuned.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os

model_path = "./llama_finetuned"
if os.path.exists(model_path):
    print("Model başarıyla kaydedilmiş!")
    print("Kayıtlı dosyalar:", os.listdir(model_path))
else:
    print("Model kaydedilmemiş, tekrar eğitmeniz gerekebilir!")


Model başarıyla kaydedilmiş!
Kayıtlı dosyalar: ['tokenizer.json', 'README.md', 'special_tokens_map.json', 'adapter_config.json', 'checkpoint-312', 'adapter_model.safetensors', 'tokenizer_config.json']


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model(model_path):
    """Eğitilmiş modeli yükler."""
    model = AutoModelForCausalLM.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer

def chat(model, tokenizer, prompt, max_length=256):
    """Kullanıcıdan gelen mesaja göre modelden yanıt üretir."""
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda" if torch.cuda.is_available() else "cpu")
    output = model.generate(input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

if __name__ == "__main__":
    model_path = "/content/drive/MyDrive/llama_finetuned_full"  # Eğittiğimiz tam model
    model, tokenizer = load_model(model_path)

    print("Chatbot'a hoş geldiniz! Çıkmak için 'exit' yazın.")
    while True:
        user_input = input("Siz: ")
        if user_input.lower() == "exit":
            break
        response = chat(model, tokenizer, user_input)
        print("Chatbot:", response)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/llama_finetuned"))


['checkpoint-312', 'tokenizer.json', 'adapter_model.safetensors', 'tokenizer_config.json', 'adapter_config.json', 'special_tokens_map.json', 'README.md']
